# Geant4 Simulaciones


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
#from statsmodels.stats.weightstats import DescrStatsW
import glob
import csv
import fileinput
from glob import glob
import re
from scipy.signal import butter, filtfilt
import scipy.integrate as spi
from scipy.stats import chi2
from scipy.signal import find_peaks

#ej de path=r'C:\Users\Sergio\Desktop\L6y7\18-04-23'
#path='C:/Users/Nicolás Molina/Desktop/L6-7/04-05-23-14kV'
#path=r'C:\Users\Sergio\Desktop\L6y7\04-05-23-19kV'

path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PL'

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')

import matplotlib.ticker as ticker


### Primero recupero ultimo PL

In [2]:
plt.close("all")
path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\MEDIDAS FINALES 28-6\pl final'

os.chdir (path)

# busco la absorbancia (en uvvis) en el punto donde excito las muestras en el pl
uvvis=glob('*.csv')

#pl=np.loadtxt("pl2fail.txt", delimiter='	', skiprows=1, unpack=True)

def mean(numbers):
    return sum(numbers)/len(numbers)


abs=[]

print(uvvis)



#plt.figure()
for film in uvvis:
  uv=np.loadtxt(film, delimiter=',', skiprows=1, unpack=True)
  lambda_=uv[0]
  intensidad=uv[1]  #corrijo
  #plt.plot(lambda_[435:445],intensidad[435:445],label=film)
  abs.append(mean(intensidad[437:443]))
#plt.legend()

#ahora si empiezo con pl

pl=np.loadtxt("pl final cef3 y nf.txt", delimiter='	', skiprows=1, unpack=True)


lambdapl=pl[0]
Iblanco=pl[1]
emcorri=pl[9]
wg280=pl[10]

Is=[]


for j in [2,3,4,5,6,7,8]:
  I0=pl[j]-Iblanco
  T=10**(abs[j-2])
  fabs=1-10**(-abs[j-2])
  Ipl=I0*emcorri*100*(1-T)/(wg280*fabs)
  Ipl=I0*emcorri*100/(wg280)
  #plt.plot(lambdapl,Ipl,label=uvvis[j-2])
  Is.append(Ipl)


max1=max(Is[4])
max2=max(Is[5])
max3=max(Is[6])

maxs=[max1,max2,max3]

Inormalizados=Is/max(maxs)


titulos2=["Ps","Ps/$CeF_{3}$ 5% ","Ps/$CeF_{3}$ 10%","Ps/NF 5%","Ps/NF 10%","Ps/NF 15%","Ps/NF 20%"]

i=0
plt.figure()
plt.title("Fotoluminiscencia de PS con diferentes cargas de $CeF_{3}$")   #Photoluminescence of PS with different loadings of Naphtalene
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad (u.a)')
for j in Inormalizados[0:3]:
  plt.plot(lambdapl[15:200],j[15:200],label=titulos2[i],marker="o",linewidth=1,ms=3)
  i=i+1
plt.legend()


plt.figure()
plt.title("Fotoluminiscencia de PS con diferentes cargas de NF")   #Photoluminescence of PS with different loadings of Naphtalene
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad (u.a)')
for j in Inormalizados[3:]:
  plt.plot(lambdapl[35:200],j[35:200],label=titulos2[i],marker="*")
  i=i+1

plt.legend()

PsCeF3_5old=Inormalizados[5]
"""
peaks, _ = find_peaks(Inormalizados[5],height=0.8)
peak_x = lambdapl[peaks]
peak_y = PsCeF3_5old[peaks]
pico1=str(round(peak_x[0]))
pico2=str(round(peak_x[1]))
#plt.scatter(peak_x[0],peak_y[0],label=f"{pico1} nm",c="red",linewidth=1.1,ls="-.",zorder=100,s=25)  
#plt.scatter(peak_x[1],peak_y[1],label=f"{pico2} nm",c="red",linewidth=1.1,ls="-.",zorder=90,s=25)  
plt.axvline(peak_x[0],ls="-.",label=f"{pico1} nm",c="red",linewidth=1)
plt.axvline(peak_x[1],ls="-.",label=f"{pico2} nm",c="blue",linewidth=1)
#plt.axvline(320,ls="-",c="black",linewidth=0.8)
#plt.axvline(340,ls="-",c="black",linewidth=0.8)
"""


Ips=Is[0]/max(Is[0])
Ips = [x if x >= 0 else -x for x in Ips]
Inf=Is[4]/max(Is[4])
Inf = [x if x >= 0 else -x for x in Inf]


h = 4.135667696e-15  # eV·s
    # Velocidad de la luz (c) en m/s
c = 3.0e8  # m/s

eV=h*c*1/lambdapl


['0 ps.csv', '1 5cef3 bis.Sample.Raw.csv', '2 10cef3 bis.Sample.Raw.csv', '3 nf 5 uvvis.csv', '4 nf 10 uvvis.csv', '5 nf 15 uvvis.csv', '6 nf 20 uvvis.csv']


### Recupero tiempos de vida

In [3]:
plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\MEDIDAS FINALES 28-6\trpl final"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

# busco la absorbancia (en uvvis) en el punto donde excito las muestras en el pl


## plots crudos
lifetime=np.loadtxt("tiempos de vida.txt",delimiter="	",skiprows=9,unpack=True)
time=lifetime[0]
nf5=lifetime[1]
pulso=lifetime[2]
ps=lifetime[3]
cef3_5=lifetime[4]
cef3_10=lifetime[5]
nf10=lifetime[6]
nf15=lifetime[7]
nf20=lifetime[8]

lifetimes=[nf5,pulso,ps,cef3_5,cef3_10,nf10,nf15,nf20]
titulos=["Ps/NF 5%","Pulso","Ps","Ps/$CeF_{3}$ 5% ","Ps/$CeF_{3}$ 10%","Ps/NF 10%","Ps/NF 15%","Ps/NF 20%"]

i=0
plt.figure()
for j in lifetimes:
    plt.plot(time,j,label=titulos[i])
    i=i+1
plt.legend()

fitps=np.loadtxt("fit ps.txt",delimiter="	",skiprows=4,unpack=True)
fit5cef3=np.loadtxt("fit 5 cef3.txt",delimiter="	",skiprows=4,unpack=True)
fit10cef3=np.loadtxt("fit 10 cef3.txt",delimiter="	",skiprows=4,unpack=True)
fit5nf=np.loadtxt("fit 5 nf.txt",delimiter="	",skiprows=4,unpack=True)
fit10nf=np.loadtxt("fit 10 nf.txt",delimiter="	",skiprows=4,unpack=True)
fit15nf=np.loadtxt("fit 15 nf.txt",delimiter="	",skiprows=4,unpack=True)
fit20nftri=np.loadtxt("fit 20 nf.txt",delimiter="	",skiprows=4,unpack=True)
fit20nftetra=np.loadtxt("fit 20 nf tetraexp.txt",delimiter="	",skiprows=4,unpack=True)


lifetimes=[ps,cef3_5,cef3_10,nf5,nf10,nf15]
fits=[fitps,fit5cef3,fit10cef3,fit5nf,fit10nf,fit15nf]
titulos=["Ps","Ps/$CeF_{3}$ 5% ","Ps/$CeF_{3}$ 10%","Ps/NF 5%","Ps/NF 10%","Ps/NF 15%"]
i=0
for j in fits:
    tiempofit=j[0]
    intfit=j[1]
    resfit=j[3]
    plt.figure()
    plt.xlabel("Tiempo (ns)")
    plt.ylabel("Cuentas (u.a)")
    plt.plot(time,lifetimes[i],label="Tiempo de vida")
    plt.plot(tiempofit,intfit,label="Ajuste trixeponencial",zorder=100,c="red")
    plt.title(titulos[i])
    plt.legend()
    i=i+1


plt.figure()
plt.plot(time,nf20,label="Tiempo de vida")
plt.plot(fit20nftetra[0],fit20nftetra[1],label="Ajuste tetraexponencial",zorder=100,linewidth=0.9,c="red")
plt.title("Ps/NF 20%")
plt.legend()
"""

ajuste=np.loadtxt("Ajuste triexp CeF3.txt",delimiter=",",skiprows=4,unpack=True)
timeajuste=ajuste[0]
intajuste=ajuste[1]

plt.plot(timeajuste,intajuste,linestyle="--",c="red",label="Ajuste triexponencial")
plt.title("Tiempo de vida de las nanopartículas de $CeF_{3}$")   #Photoluminescence of PS with different loadings of Naphtalene
plt.xlabel('Tiempo (ns)')
plt.ylabel('Intensidad (u.a)')
plt.legend()

"""

lifetimes3=[ps,cef3_10,nf10]   #
titulos3=["Ps","Ps/$CeF_{3}$ 10% ","Ps/NF 10%"]   #
i=0 
plt.figure()
plt.title("Tiempos de vida de centelladores")
plt.xlabel("Tiempo (ns)")
plt.ylabel("Cuentas (u.a)")
for j in lifetimes3:
    plt.plot(time,j,label=titulos3[i],linewidth=0.9,zorder=-1)
    i=i+1
plt.legend()


plt.figure()
plt.title("Pulso de excitación")
plt.xlabel("Tiempo (ns)")
plt.ylabel("Cuentas (u.a)")
plt.plot(time,pulso)
plt.legend()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


## Comparo simulaciones con experimental


In [9]:
plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\simulaciones"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

emision=np.loadtxt("prueba emision.txt",delimiter="	",skiprows=0,unpack=True)
lifetime=np.loadtxt("prueba lifetime.txt",delimiter="	",skiprows=0,unpack=True)

wlemision=emision[0]/1000
Ipssim=emision[1]/max(emision[1])



plt.figure()
plt.title("Espectro de emisión del poliestireno (Ps)")
plt.plot(lambdapl[15:],Ips[15:],label="Experimental")
plt.plot(wlemision,Ipssim,label="Simulación Geant4",zorder=100)
plt.xlabel("$\lambda$ (nm)")
#plt.ylabel("Cuentas (u.a)")
plt.legend()

In [10]:
plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\simulaciones"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

emision=np.loadtxt("prueba emision nf.txt",delimiter="	",skiprows=0,unpack=True)
#lifetime=np.loadtxt("prueba lifetime nf.txt",delimiter="	",skiprows=0,unpack=True)

wlemision=emision[0]/1000
Infsim=emision[1]/max(emision[1])


plt.figure()
plt.title("Espectro de emisión del Naftaleno (Nf)")
plt.plot(lambdapl[15:],Inf[15:],label="Experimental", linewidth=3, linestyle='-')
plt.plot(wlemision,Infsim,label="Simulación Geant4",zorder=100)
plt.xlabel("$\lambda$ (nm)")
#plt.ylabel("Cuentas (u.a)")
plt.legend()

### alpha Arazi

In [8]:
plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\simulaciones"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

emision=np.loadtxt("PsEmision1000AlphaArazi.txt",delimiter="	",skiprows=0,unpack=True)
#lifetime=np.loadtxt("prueba lifetime nf.txt",delimiter="	",skiprows=0,unpack=True)

wlemision=emision[0]/1000
Ipssim=emision[1]


plt.figure()
plt.title(r"Centelladores expuestos a fuente $\alpha$ durante 0.25s")
#plt.plot(lambdapl[15:],Inf[15:],label="Experimental", linewidth=3, linestyle='-')
plt.plot(wlemision,Ipssim,zorder=100,label="Ps")
plt.xlabel("$\lambda$ (nm)")
#plt.ylabel("Cuentas (u.a)")



####


emision=np.loadtxt("NfEmision1000AlphaArazi.txt",delimiter="	",skiprows=0,unpack=True)
#lifetime=np.loadtxt("prueba lifetime nf.txt",delimiter="	",skiprows=0,unpack=True)

wlemision=emision[0]/1000
Infsim=emision[1]


#plt.figure()
#plt.title("Nf expuesto a fuente alpha por 1s simulado")
#plt.plot(lambdapl[15:],Inf[15:],label="Experimental", linewidth=3, linestyle='-')
plt.plot(wlemision,Infsim,zorder=100,label="Ps/Nf 10$\%$")
plt.xlabel("$\lambda$ (nm)")
plt.ylabel("Cuentas (u.a)")
plt.legend()


######### TIEMPOS DE VIDA

tps,cuentasps=np.loadtxt("PsLT1000AlphaArazi.txt",delimiter="	",skiprows=0,unpack=True)
tnf,cuentasnf=np.loadtxt("NfLT1000AlphaArazi.txt",delimiter="	",skiprows=0,unpack=True)


plt.figure()
#plt.title(r"Tiempo de vida del Ps simulado")
plt.title(r"Tiempos de vida de centelladores expuestos a fuente $\alpha$ durante 0.25s")
#plt.plot(lambdapl[15:],Inf[15:],label="Experimental", linewidth=3, linestyle='-')
plt.plot(tps,cuentasps/max(cuentasps),zorder=100,label="Ps")
plt.xlabel("t (s)")
#plt.ylabel("Cuentas (u.a)")

#plt.figure()
#plt.title(r"Tiempo de vida del Nf simulado")
plt.plot(tnf,cuentasnf/max(cuentasnf),zorder=100,label="Ps/Nf 10$\%$")
#plt.ylabel("Cuentas (u.a)")
plt.xlabel("t (s)")
plt.legend()


## POSTER

In [25]:
plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\simulaciones\poster"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

pvt08 =np.loadtxt("PVT0.8mm.txt",delimiter="	",skiprows=0,unpack=True)
pvt2=np.loadtxt("PVT2mm.txt",delimiter="	",skiprows=0,unpack=True)
ps2=np.loadtxt("Ps2mm.txt",delimiter="	",skiprows=0,unpack=True)
psnf=np.loadtxt("psnf2mm.txt",delimiter="	",skiprows=0,unpack=True)

pls=[ps2,pvt08]
titulos=["Ps de 2 mm de espesor","PVT de 0,8 mm de espesor"]

norm=max(psnf[1])

j=0
plt.figure()
plt.title("Fotoluminiscencia simulada con GEANT4")
for pl in pls:
    plt.plot(pl[0]*1000000,pl[1]/norm,label=titulos[j],linewidth=2)

    j=j+1
plt.xlabel("$\lambda$ (nm)")
plt.plot(pvt2[0]*1000000,pvt2[1]/norm,label="PVT de 2 mm de espesor",c="purple",linewidth=2)
plt.ylabel("Intensidad (u.a)")
plt.legend()

plt.figure()
plt.title("Fotoluminiscencia simulada con GEANT4")
plt.plot(pvt2[0]*1000000,pvt2[1]/norm,label="PVT de 2 mm de espesor",c="purple",linewidth=2)
plt.plot(psnf[0]*1000000,psnf[1]/norm,label="Ps/Nf de 2 mm de espesor",c="black",linewidth=2)
plt.ylabel("Intensidad (u.a)")
plt.xlabel("$\lambda$ (nm)")
plt.legend()

In [7]:

h = 4.135667696e-15  # eV·s
    # Velocidad de la luz (c) en m/s
c = 3.0e8  # m/s



plt.close("all")
path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\cosas que no uso"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

nm,int=np.loadtxt("emision pvt.txt",delimiter="	",skiprows=0,unpack=True)



eV=h*c*1/(nm*1.0e-9)
eV=1240/nm